# Below: Composite trapezoidal, Simpson, and Romberg integrators.

In [1]:
import numpy as np

Here, we apply the trapezoidal rule. Observe that as $h \to \frac{h}{2}$, we keep all of the previous points and add points halfway between. Hence, we need only compute half the function values every time we decrease $h$.

In [7]:
def compute_trapezoidal_integral(a,b,f,tol=1.e-6):
    Function_Evals = 0
    n = 1
    Ts = []
    hs = []
    mainDifs =[]
    fsPrev = None
    fa = f(a)
    fb= f(b)
    Function_Evals += 2
    

    def get_T(n,fsPrev=None):
        nonlocal Function_Evals, fa, fb

        h=(b-a)/n
        adjustor = h*(fa+fb)/2
        hs.append(h)
        if fsPrev is None:
            fsPrev = np.array([fa,fb])
        
        # we would like n to run from 1->2->4->8  for simplicity
        # observe that each list has n+1 points
        # already have the first and last vals f(a) and f(b)
        # we also don't want to evaluate any of the previous points, which would be at a + 2kh for natural numbers k, as we are dividing k by 2
        # this is i = 2, 4, 6, ...
        # we also throw out the i=0 point (a) and the i = n (b) points
        # so we want our final array to have i = 1,3,5,...

        ivals = np.arange(1,n,2)
        xis = a+ivals*h
        Function_Evals += len(ivals)
        fsPrev = np.concatenate((fsPrev,f(xis)))
        T = h*np.sum(fsPrev)-adjustor
        return(T,fsPrev)

    while len(Ts) < 2 or (np.abs((Ts[-1]-Ts[-2])/Ts[-1]) > tol):
        T,fsPrev = get_T(n,fsPrev)
        Ts.append(T)
        if len(Ts) >1:
            mainDifs.append(np.abs((Ts[-1]-Ts[-2])/Ts[-1]))
        n= 2*n
    hs = np.array(hs)
    Ts = np.array(Ts)



    out = np.column_stack((hs[1:], Ts[1:], np.array(mainDifs)))


    return(out,Function_Evals)

Here, we do the same as above but for the 'simpson' rule. Observe that as n->2n, an old point of the form $a+(i+0.5)h_{old}$ now looks like $a+(i+0.5)2h_{new} = a+(2i+1)h_{new}$ for $i < \frac{n_{new}}{2}$; that is, it's one of the new points of the form $a+ih$ for $i < n_{new}$. In fact, this is clearly a bijection; each new point of the form $a+ih_{new}$ is one of the previous step's points of the form $a+(i+.5)h_{old}$. Hence, we never need to compute these 'whole-number' interpolants.

Hence, we need only add in the interpolation points of the form $a+(i+0.5)2h_{new}$. At the $nth$ step, there are $n$ of these. See code below. 


In [ ]:
def compute_Simpson_integral(a,b,f,tol=1.e-6):
    Function_Evals = 0
    n = 1
    Ss = []
    hs = []
    mainDifs = []
    fsPrev = None
    fa = f(a)
    fb= f(b)
    Function_Evals += 2


    def get_S(n,fsPrev=None):
        nonlocal Function_Evals, fa, fb

        h=(b-a)/n
        adjustor = h*(fa+fb)/6
        hs.append(h)
        if fsPrev is None:
            fsPrev = np.array([fa,fb])

        ivals = np.arange(0,n,1)
        xis = a+(ivals+0.5)*h
        Function_Evals += len(ivals)
        fxis = f(xis)
        S= (2*h/3)*np.sum(fxis) + (h/3)*np.sum(fsPrev) - adjustor 
        fsPrev = np.concatenate((fsPrev,fxis))

        return(S,fsPrev)

    while len(Ss) < 2 or (np.abs((Ss[-1]-Ss[-2])/Ss[-1]) > tol):
        S,fsPrev = get_S(n,fsPrev)
        Ss.append(S)
        if len(Ss) >1:
            mainDifs.append(np.abs((Ss[-1]-Ss[-2])/Ss[-1]))
        n= 2*n
    hs = np.array(hs)
    Ss = np.array(Ss)


    out = np.column_stack((hs[1:], Ss[1:], np.array(mainDifs)))

    return(out,Function_Evals)


Here, we compute apply Romberg integration. First, fill out the first column of the romberg table with our standard trapezoidal rule. This corresponds to the first line at the bottom of slide 8 on the integration_extrapolation slides. Then, forward fill the rest of our romberg table accordingly. See below.

In [ ]:
def compute_Romberg_integral(a,b,f,tol=1.e-6):
    Function_Evals = 0
    n = 1
    Ts = []
    Romberg_Table = []
    hs = []
    mainDifs =[]
    fsPrev = None
    fa = f(a)
    fb= f(b)
    Function_Evals += 2
    
    # this is identical to our previous trapezoidal rule
    def get_T(n,fsPrev=None):
        nonlocal Function_Evals, fa, fb

        h=(b-a)/n
        adjustor = h*(fa+fb)/2
        hs.append(h)
        if fsPrev is None:
            fsPrev = np.array([fa,fb])
        
        # we would like n to run from 1->2->4->8  for simplicity
        # observe that each list has n+1 points
        # already have the first and last vals f(a) and f(b)
        # we also don't want to evaluate any of the previous points, which would be at a + 2kh for natural numbers k, as we are dividing k by 2
        # this is i = 2, 4, 6, ...
        # we also throw out the i=0 point (a) and the i = n (b) points
        # so we want our final array to have i = 1,3,5,...

        ivals = np.arange(1,n,2)
        xis = a+ivals*h
        Function_Evals += len(ivals)
        fsPrev = np.concatenate((fsPrev,f(xis)))
        T = h*np.sum(fsPrev)-adjustor
        return(T,fsPrev)
    
    def adjust_Romberg(table,ts):

        # here, we apply the algorithm from slide 8 of the integration_extrapolation slides
        n = len(ts)
        table.append([])
        table[-1].append(ts[-1])
        if n > 1:
            for j in range(1,n):
                frac = (table[-1][j-1]-table[-2][j-1])/(4**(j) -1)
                table[-1].append(table[-1][-1]+frac)
        return(table)

    # check convergence with romberg rule 

    while len(Ts) < 2 or (np.abs((Romberg_Table[-1][-2]-Romberg_Table[-1][-1])/Romberg_Table[-1][-1]) > tol):
                    

        T,fsPrev = get_T(n,fsPrev)


        Ts.append(T)
        # if Romberg_Table == []:
        #     for T in Ts:
        #         Romberg_Table.append([T])
        Romberg_Table = adjust_Romberg(Romberg_Table,Ts)

        if len(Ts) >1:
            mainDifs.append(np.abs((Ts[-1]-Ts[-2])/Ts[-1]))
        n= 2*n



    return(Romberg_Table,Function_Evals)

Here, we test our integration methods.

In [5]:
# want to integrate this from 0 to 3
def fFrac(x):
    return(x/(1+x**2))

# want to integrate from 0 to .95
def fHyper(x):
    return(1/(1-x))

# want to int from 0 to np.pi/2 for m=.5,.8,.95
def get_SinM(m):
    def SinM(x):
        return(1/(np.sqrt(1-m*(np.sin(x)**2))))
    return(SinM)

In [6]:
print("#######################################################################################")
for func,formula,interval,trueVals in zip([fFrac,fHyper,get_SinM(.5),get_SinM(.8),get_SinM(.95)],['x/1+x^2','1/(1-x)','1/sqrt(1-.5sin^2(x))','1/sqrt(1-.8sin^2(x))','1/sqrt(1-.95sin^2(x))'],
                                          [[0,3],[0,.95],[0,np.pi/2],[0,np.pi/2],[0,np.pi/2]],[.5*np.log(10),np.log(20),None,None,None]):
    print(f"f(x) = {formula}, [a,b] = {interval}")
    print(f"True Value of Integral = {trueVals if trueVals else 'Not Provided'}")
    outTrap,evalsTrap = compute_trapezoidal_integral(interval[0],interval[1],func)
    print("Composite Trapezoidal Rule")
    Ts = outTrap[:,1]
    Difs = outTrap[:,2]
    print(f"Approximate Value of Integral = {Ts[-1]}")
    # print(f"Absolute Error = {np.abs(Ts[-1]-trueVals) if trueVals else 'True Value Not Provided'}")
    # print(f"Relative Error = {np.abs((Ts[-1]-trueVals)/trueVals) if trueVals else 'True Value Not Provided'}")
    print(f"Number of Evals = {evalsTrap}\n\n")
    outSimp,evalsSimp = compute_Simpson_integral(interval[0],interval[1],func)
    print("Composite Simpson Rule")
    Ss = outSimp[:,1]
    DifsS = outSimp[:,2]
    print(f"Approximate Value of Integral = {Ss[-1]}")
    # print(f"Absolute Error = {np.abs((Ss[-1]-trueVals)) if trueVals else 'True Value Not Provided'}")
    # print(f"Relative Error = {np.abs((Ss[-1]-trueVals)/trueVals) if trueVals else 'True Value Not Provided'}")
    print(f"Number of Evals = {evalsSimp}\n\n")
    print(f"Romberg Integral")
    rTable,rEvals = compute_Romberg_integral(interval[0],interval[1],func,tol=1.e-6)
    print(f"Approximate Value of Integral = {rTable[-1][-1]}")
    print(f"Number of Evals = {rEvals}")
    print("#######################################################################################")

    continue

#######################################################################################
f(x) = x/1+x^2, [a,b] = [0, 3]
True Value of Integral = 1.151292546497023
Composite Trapezoidal Rule
Approximate Value of Integral = 1.1512923533779356
Number of Evals = 2049


Composite Simpson Rule
Approximate Value of Integral = 1.151292556540326
Number of Evals = 129


Romberg Integral
Approximate Value of Integral = 1.1512927361761305
Number of Evals = 33
#######################################################################################
f(x) = 1/(1-x), [a,b] = [0, 0.95]
True Value of Integral = 2.995732273553991
Composite Trapezoidal Rule
Approximate Value of Integral = 2.995732720709657
Number of Evals = 8193


Composite Simpson Rule
Approximate Value of Integral = 2.995732336561578
Number of Evals = 513


Romberg Integral
Approximate Value of Integral = 2.9957721108751363
Number of Evals = 65
#######################################################################################
f(x) = 1